# Day 07

Imports.

In [1]:
from collections import defaultdict


Read input.

In [2]:
with open("07_input.txt", "r") as f:
    puzzle_input = f.read().splitlines()


Define data structures and utilities.

In [3]:
class Hand:
    card_strength : dict[str, int] = {
        card: strength
        for strength, card in enumerate("23456789TJQKA")
    }

    def __init__(self, cards: str, bid: str, jokers: bool = False):
        self.cards         : str                   = cards
        self.bid           : int                   = int(bid)
        self.unique_cards  : set                   = set(cards)
        self.n_unique_cards: int                   = len(self.unique_cards)
        self.card_counts   : defaultdict[str, int] = self._cnt_unique_cards()

        if jokers and "J" in cards:
            self._account_for_jokers()

        self.type          : str                   = self._get_type()
        self.type_score    : int                   = self._calc_type_score()

    def _account_for_jokers(self) -> None:
        n_jokers = self.card_counts.pop("J")
        if n_jokers == 5:

            self.card_counts["A"] = 5
        else:
            self.n_unique_cards -= 1
            ties = len(set(self.card_counts.values())) == 1

            if ties:
                best_card = max(self.card_counts, key=self.card_strength.get)
            else:
                best_card = max(self.card_counts, key=self.card_counts.get)

            self.card_counts[best_card] += n_jokers

    def _cnt_unique_cards(self) -> defaultdict[str, int]:
        card_counts = defaultdict(int)
        for card in self.cards:
            card_counts[card] += 1
        return card_counts

    def _get_type(self) -> str:
        match self.n_unique_cards:
            case 5:
                hand_type = "high_cards"
            case 4:
                hand_type = "one_pair"
            case 3:
                if 2 in self.card_counts.values():
                    hand_type = "two_pairs"
                else:
                    hand_type = "three_of_a_kind"
            case 2:
                if 2 in self.card_counts.values():
                    hand_type = "full_house"
                else:
                    hand_type = "four_of_a_kind"
            case 1:
                hand_type = "five_of_a_kind"
        return hand_type

    def _calc_type_score(self) -> int:
        n_cards_per_hand = 5
        n_all_unique_cards = 13
        weights = [n_all_unique_cards ** w for w in range(n_cards_per_hand, 0, -1)]

        score = 0
        for card, w in zip(self.cards, weights):
            score += w * self.card_strength[card]

        return score


def parse_input(
    puzzle_input: list[str], jokers: bool = False
) -> dict[str, list[Hand]]:
    hand_types = [
        "high_cards",
        "one_pair",
        "two_pairs",
        "three_of_a_kind",
        "full_house",
        "four_of_a_kind",
        "five_of_a_kind",
    ]
    hands_by_type = {type_i: [] for type_i in hand_types}

    for line in puzzle_input:
        cards, bid = line.split()
        hand = Hand(cards=cards, bid=bid, jokers=jokers)
        hands_by_type[hand.type].append(hand)

    return hands_by_type


def calc_total_winnings(hands_by_type: dict[str, list[Hand]]) -> int:
    total_winnings, rank = 0, 0
    for type_i_hands in hands_by_type.values():
        type_i_hands.sort(key=lambda hand: hand.type_score)

        for hand in type_i_hands:
            rank += 1
            total_winnings += rank * hand.bid

    return total_winnings


## Part 1

In [4]:
%%timeit

hands_by_type = parse_input(puzzle_input=puzzle_input)
total_winnings = calc_total_winnings(hands_by_type=hands_by_type)
assert total_winnings == 248_422_077


1.97 ms ± 19.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Part 2

In [5]:
%%timeit

Hand.card_strength = {
    card: strength
    for strength, card in enumerate("J23456789TQKA")
}

hands_by_type = parse_input(puzzle_input=puzzle_input, jokers=True)
total_winnings = calc_total_winnings(hands_by_type=hands_by_type)
assert total_winnings == 249_817_836


2.22 ms ± 8.01 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
